<a href="https://colab.research.google.com/github/Tensai-Pro/SRI-PracticeTasks/blob/master/Word%20Embeddings/3)%20fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import pandas as pd
import numpy as np

from gensim.utils import simple_preprocess
from gensim.models import FastText

import nltk;
nltk.download('stopwords');
from nltk.corpus import stopwords;

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import fasttext

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
df = pd.read_csv('/content/gdrive/MyDrive/Practice Files/train.zip')
df.head(3)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...


# Предобработка и подготовка для fastText

In [22]:
stopwords = stopwords.words('english')

In [23]:
def prepare_text(text):
  tokens = simple_preprocess(text)
  tokens = [tok for tok in tokens if tok not in stopwords]
  return ' '.join(tokens)

In [24]:
df['clean_text'] = df.Description.apply(prepare_text)
df.head()

,Class Index,Title,Description,clean_text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",reuters short sellers wall street dwindling ba...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,reuters private investment firm carlyle group ...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,reuters soaring crude prices plus worries econ...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,reuters authorities halted oil export flows ma...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",afp tearaway world oil prices toppling records...


In [25]:
X_train, X_test, y_train, y_test = train_test_split(df.clean_text, df['Class Index'],
                                                    shuffle=True, random_state=42)

In [26]:
with open('train_data.txt', 'w', encoding='utf-8') as f:
  for i in range(len(X_train)):
    f.write('__label__' + str(y_train.values[i]) + ' ' + X_train.values[i] + '\n')

with open('test_data.txt', 'w', encoding='utf-8') as f:
  for i in range(len(X_test)):
    f.write('__label__' + str(y_test.values[i]) + ' ' + X_test.values[i] + '\n')

# fastText

## Библиотека fastTest

In [40]:
model = fasttext.train_supervised(input='train_data.txt', epoch=50, wordNgrams=2,
                                  lr=0.005)

In [41]:
result = model.test('test_data.txt')

In [42]:
print('Test Accuracy:', result)

Test Accuracy: (30000, 0.9055333333333333, 0.9055333333333333)


In [43]:
model.predict(X_test.values[10])

(('__label__4',), array([0.92115545]))

In [44]:
y_test.values[10]

4

## Модуль gensim fastText

In [57]:
clean_text_tokens = X_train.apply(lambda x: x.split(' '))

In [59]:
model_gensim = FastText(clean_text_tokens, vector_size=100, window=5,
                        min_count=5, workers=4)

In [60]:
model_gensim.wv['gastroenteritis']

array([-0.23577021, -0.10425048, -0.24582063, -0.13068034,  0.14536987,
       -0.24623318,  0.07570754, -0.27244994,  0.4126305 , -0.3626068 ,
       -0.46280134, -0.30879936, -0.18818915,  0.2854791 ,  0.48376396,
       -0.44618478,  0.27850464, -0.12138075, -0.40938804, -0.3645964 ,
       -0.67231804,  0.01136128, -0.27266   ,  0.37936842,  0.20974083,
       -0.6175861 ,  0.45635837, -0.00381344,  0.5292319 ,  0.53353745,
       -0.08043919, -0.15133677,  0.1988649 , -0.23306853, -0.00418355,
        0.15871362,  0.09813847, -0.27082092, -0.41767123, -0.778806  ,
        0.10307579,  0.02591626,  0.4904344 ,  0.02858465,  0.47668988,
        0.13007371,  0.24554306, -0.06656601,  0.35808712, -0.04863928,
        0.30036712,  0.32941318, -0.24394122,  0.21496522, -0.04596694,
       -0.6290439 , -0.00813384,  0.02106195,  0.26692462,  0.38001427,
       -0.42396203,  0.20172271, -0.24896508, -0.35892183,  0.73892367,
        0.29539588, -0.23748218,  0.15338038,  0.30432683,  0.59

In [65]:
model_gensim.wv.most_similar('castle')

[('hustle', 0.7908443212509155),
 ('nestle', 0.7807610630989075),
 ('newcastle', 0.7516541481018066),
 ('castillo', 0.710847020149231),
 ('carlisle', 0.7069178819656372),
 ('whistle', 0.7023760080337524),
 ('winchester', 0.687375009059906),
 ('eastley', 0.6839461326599121),
 ('turtle', 0.6782268285751343),
 ('uncasville', 0.6753606796264648)]

# Векторизация

In [66]:
def vectorize(text):
    vectorized_text = np.array([model_gensim.wv[word] for word in text if word in model_gensim.wv])
    if len(vectorized_text) == 0:
        return np.zeros(100)
    return vectorized_text.mean(axis=0)

In [68]:
X_train_vectorized = np.array([vectorize(sentence) for sentence in X_train])
X_test_vectorized = np.array([vectorize(sentence) for sentence in X_test])

In [70]:
X_train_vectorized.shape

(90000, 100)

# Классификация

In [73]:
scaler = MinMaxScaler()

In [74]:
X_train_scaled = scaler.fit_transform(X_train_vectorized)
X_test_scaled = scaler.transform(X_test_vectorized)

In [75]:
clf_nb = MultinomialNB().fit(X_train_scaled, y_train)

In [76]:
y_pred_nb = clf_nb.predict(X_test_scaled)

In [77]:
acc_score = clf_nb.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 28.163333333333334%
